## <center> Chase Case </center>

### Import Data and Storage in Hash Table

In [1]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
#Variables
FLAG_TABLE = False

In [3]:
path = os.getcwd()
folder = "output_chase_documentai"
folder_file = "file_1"
files = os.listdir(os.path.join(folder, folder_file))

In [4]:
dic_data = {}
for file in files:
    data = pd.read_csv(os.path.join(path, folder, folder_file, file))
    p = "(pg\d\w+)"
    regex_p= re.compile(p)
    dic_key = regex_p.search(file).group()
    dic_data[dic_key] = data

In [5]:
for key in dic_data.keys():
    print(key)

pg1_tb0
pg1_tb1
pg1_tb2
pg2_tb0
pg3_tb0
pg3_tb1
pg4_tb0
pg4_tb1
pg4_tb2
pg5_tb0
pg6_tb0
pg6_tb1
pg7_tb0
pg7_tb1
pg7_tb2
pg7_tb3
pg8_tb0
pg8_tb1


In [9]:
dic_data["pg3_tb1"]#.iloc[0:1].values

,DATE,DESCRIPTION,AMOUNT
0,03/29,Orig CO Name: Square Inc Descr:210329P2 Sec:CCD,91.15 Orig ID:9424300002 Desc Date: 210329 CO ...
1,NaN,ID:L205609880756 L65480 Trn: 0889288526Tc,Ind Name:Miami Virgin Hair Corp
2,03/29,Orig CO Name: Square Inc Descr:210329P2 Sec:CC...,65.59 Orig ID:9424300002 Desc Date:210329 CO E...
3,03/29,Zelle Payment From Lorena,Tubias Bacv67LA34Q7 30.00
4,03/31,ATM Cash Deposit,03/31 19495 Biscayne Blvd Aventura FL Card 844...
5,03/31,Orig CO Name: Square Inc,Orig ID:9424300002 Desc Date: 210331 CO Entry ...
6,NaN,Descr:210331P2 Sec: CCD ID:L205610393262 L6552...,Trace#:021000028975194 Eed: 210331 Ind Ind Nam...
7,Total Deposits,and Additions,"$24,809.90"


### Separate used table and Storage in Another Dic

In [7]:
tables = {}
for key in dic_data.keys():
    if len(dic_data[key].columns) >= 3 and len(dic_data[key].columns) < 6:
        for column in dic_data[key].columns:
            if "DATE" in column:
                FLAG_TABLE = True
    if FLAG_TABLE:
        tables[key] = dic_data[key]
        FLAG_TABLE = False

In [8]:
#Nota: hacerlo de esta forma, implica arregalr antes tema clumnas en el header

# tables = {}
# for key in dic_data.keys():
#     if len(dic_data[key].columns) >= 3 and len(dic_data[key].columns) < 6 and "DATE" in dic_data[key].columns:
#         tables[key] = dic_data[key]


In [9]:
for key in tables.keys():
    print(key)

pg1_tb0
pg2_tb0
pg2_tb1
pg2_tb2


In [10]:
tables["pg2_tb2"] = tables["pg2_tb2"].rename(columns={"Unnamed: 2":"column_not_name"})
tables["pg2_tb0"] = tables["pg2_tb0"].rename(columns={"Unnamed: 2":"column_not_name"})

### Preprocessing tables "column_not_name"

In [11]:
tables_2 = {}
for key in tables.keys():
    if "column_not_name" in tables[key].columns and len(tables[key].columns) == 4:
        
        table = tables[key]
        
        table["DESCRIPTION"] = table.apply(lambda x: x["DESCRIPTION"] + (x.column_not_name)
                                            if pd.notnull(x.column_not_name)
                                            else x["DESCRIPTION"],
                                            axis=1)
        
        table = table.drop("column_not_name",axis=1)
        tables_2[key] = table

    elif "column_not_name" in tables[key].columns and len(tables[key].columns) == 3:
        
        table = tables[key].astype(str)
        patron = "(\d*|\d{1,3}(,\d{3})*)(\.\d+)"
        regex_number_extractor = re.compile(patron)
        table["AMOUNT"] = table["AMOUNT DESCRIPTION"].apply(lambda x: regex_number_extractor.search(x))
        table["AMOUNT"] = table["AMOUNT"].apply(lambda x: x if x is None else x.group())
        
        table["AMOUNT"] = table.apply(lambda x: x["AMOUNT"] 
                                      if pd.notnull(x["AMOUNT"])
                                      else x.column_not_name,
                                      axis=1)
        
        table = table.rename(columns={"AMOUNT DESCRIPTION":"DESCRIPTION"})
        table["DESCRIPTION"] = table.apply(lambda x: x["DESCRIPTION"].replace(x["AMOUNT"], ""),axis=1)
        table["DESCRIPTION"] = table.apply(lambda x: x["DESCRIPTION"].replace("$", ""),axis=1)
        table = table.drop("column_not_name",axis=1)
        tables_2[key] = table
        
    else:
        tables_2[key] = tables[key]

In [12]:
for key in tables_2.keys():
    print(key)

pg1_tb0
pg2_tb0
pg2_tb1
pg2_tb2


In [13]:
tables_2["pg1_tb0"]

,DATE 08/02,DESCRIPTION Zelle Payment From Sergio Molho Bacuco6Dhrcf,AMOUNT $229.00
0,08/02,Zelle Zelle Payment From Ignacio Lanfranconi B...,100.0
1,08/03,Orig CO Name:Venmo Orig ID:5264681992 Desc Dat...,240.0
2,08/05,Zelle Payment From Dina Eidman Tdp0Gdd6G7G2,50.0
3,08/09,Remote Online Deposit 1,60.0
4,08/09,Orig CO Name:Venmo Orig ID:5264681992 Desc Dat...,107.0
5,08/09,Orig CO Name:Venmo Orig ID:5264681992 Desc Dat...,90.0
6,08/10,Zelle Payment From Hernan Berman Bacvkdsxl2Rv,80.0
7,08/20,Orig CO Name:Venmo Orig ID:5264681992 Desc Dat...,150.0
8,08/23,Zelle Payment From Natalia Palazzo Wfct0C7Nthjl,115.0
9,08/23,Zelle Payment From Layla De Armas 12442941880,55.0


In [14]:
#Check logica de agregado de columna del titulo al DF

# df = tables_2["pg1_tb0"].copy(deep=True)
# df.columns.values
# test = [x.replace("DATE","").strip() if "DATE" in x 
#         else x.replace("DESCRIPTION","").strip() if "DESCRIPTION" in x 
#         else x.replace("AMOUNT","").strip() if "AMOUNT" in x 
#         else x 
#         for x in df.columns.values]
# df.loc[-1] = test  # adding a row
# df.index = df.index + 1  # shifting index
# df = df.sort_index()
# df 

In [15]:
#ESTE CAMPO HAY QUE RECONSTRUIRLO COMPLETO

tables_2["pg2_tb2"].DESCRIPTION[1] 

'Orig CO Name:Venmo Orig ID:5264681992 Descr:Cashout Sec:PPD Trace#:091000018706737 EedInd Name:Crumbs Crumbles Trn: 2438706737TCDesc Date:210830 CO Entry :210831 Ind ID:'

### Indentificate Tables with Income and Withdrawals

In [16]:
tables_3 = {}
for key in tables_2.keys():
    if "DATE" in tables_2[key].columns and \
    "Additions" in tables_2[key].DESCRIPTION.values[len(tables_2[key]) - 1]:
        tables_2[key]["type"] = "Income"
        tables_2[key].drop(tables_2[key].index[len(tables_2[key])-1],inplace=True)
        tables_3[key] = tables_2[key]
        
    elif "DATE" in tables_2[key].columns and \
    "Withdrawals" in tables_2[key].DESCRIPTION.values[len(tables_2[key]) - 1]:
        tables_2[key]["type"] = "Withdrawals"
        tables_2[key].drop(tables_2[key].index[len(tables_2[key])-1],inplace=True)
        tables_3[key] = tables_2[key]
        
    else:
        if len(tables_2[key].columns) == 4:
            row = [x.replace("DATE","").strip() if "DATE" in x 
                    else x.replace("DESCRIPTION","").strip() if "DESCRIPTION" in x 
                    else x.replace("AMOUNT","").strip() if "AMOUNT" in x 
                    else x 
                    for x in tables_2[key].columns.values]
            tables_2[key].loc[-1] = row  # adding a row
            tables_2[key].index = tables_2[key].index + 1  # shifting index
            tables_2[key] = tables_2[key].sort_index()
            tables_2[key].columns = ["DATE", "DESCRIPTION", "column_not_name", "AMOUNT"]
            tables_2[key]["DESCRIPTION"] = tables_2[key].DESCRIPTION + " " + \
                                           tables_2[key].column_not_name
            tables_2[key] = tables_2[key].drop("column_not_name",axis=1)
            tables_2[key]["type"] = "Withdrawals"
            tables_3[key] = tables_2[key]
        else:
            row = [x.replace("DATE","").strip() if "DATE" in x 
                    else x.replace("DESCRIPTION","").strip() if "DESCRIPTION" in x 
                    else x.replace("AMOUNT","").strip() if "AMOUNT" in x 
                    else x 
                    for x in tables_2[key].columns.values]
            tables_2[key].loc[-1] = row  # adding a row
            tables_2[key].index = tables_2[key].index + 1  # shifting index
            tables_2[key] = tables_2[key].sort_index()
            tables_2[key].columns = ["DATE", "DESCRIPTION","AMOUNT"]
            tables_2[key]["type"] = "Income"
            tables_3[key] = tables_2[key]
            

In [17]:
for key in tables_3.keys():
    print(key)

pg1_tb0
pg2_tb0
pg2_tb1
pg2_tb2


In [18]:
tables_3["pg1_tb0"]

,DATE,DESCRIPTION,AMOUNT,type
0,08/02,Zelle Payment From Sergio Molho Bacuco6Dhrcf,$229.00,Income
1,08/02,Zelle Zelle Payment From Ignacio Lanfranconi B...,100.0,Income
2,08/03,Orig CO Name:Venmo Orig ID:5264681992 Desc Dat...,240.0,Income
3,08/05,Zelle Payment From Dina Eidman Tdp0Gdd6G7G2,50.0,Income
4,08/09,Remote Online Deposit 1,60.0,Income
5,08/09,Orig CO Name:Venmo Orig ID:5264681992 Desc Dat...,107.0,Income
6,08/09,Orig CO Name:Venmo Orig ID:5264681992 Desc Dat...,90.0,Income
7,08/10,Zelle Payment From Hernan Berman Bacvkdsxl2Rv,80.0,Income
8,08/20,Orig CO Name:Venmo Orig ID:5264681992 Desc Dat...,150.0,Income
9,08/23,Zelle Payment From Natalia Palazzo Wfct0C7Nthjl,115.0,Income


### Final Step and Compile all Information in 1 DataFrame

In [19]:
df_chase_final = pd.DataFrame()
for key in tables_3.keys():
    df_chase_final = pd.concat([df_chase_final, tables_3[key]], axis=0)
    df_chase_final = df_chase_final.sort_values(["type", "DATE"])
    df_chase_final.AMOUNT = df_chase_final.AMOUNT.astype(str)
    df_chase_final.AMOUNT = df_chase_final.AMOUNT.apply(lambda x: x.replace("$", ""))
    df_chase_final.AMOUNT = df_chase_final.AMOUNT.astype(float)
    

### Check Information

In [20]:
df_chase_final[df_chase_final.type == "Income"].AMOUNT.sum()

2130.0

In [21]:
df_chase_final[df_chase_final.type == "Withdrawals"].AMOUNT.sum()

1898.6800000000003

## Delete Names

In [22]:
names = ["Sergio Molho",
"Ignacio Lanfranconi",
"Dina Eidman",
"Hernan Berman",
"Natalia Palazzo",
"Layla De Armas",
"Natalia Palazzo",
"Brian Hunis",
"Natalia Wartensleben",
"Juan Arango Dulce De Lecehe",
"Maia",
"Cynthia Romina",
"Venmo"]

In [23]:
for name in names:
    df_chase_final.DESCRIPTION = df_chase_final.DESCRIPTION.apply(lambda x: x.replace(name, "#####"))

## Save Output

In [24]:
df_chase_final["bank"] = "Chase"
folder_output = "output_script"
file = "chase_output.csv"

In [25]:
df_chase_final.to_csv(os.path.join(path,folder_output, file), index=False, encoding="utf8")